Name: Bowei Dong, Ruiyan Song

* **Instructions:**

`1` put the datafile input as 'kenken.txt' file in the current working dircetory; Currently, 'kenken.txt' contains the example 6x6 puzzle in the homework pdf. We also have three other kenken puzzles available for testing:'kenken2.txt' (5x5), 'kenken3.txt' (4x4), 'kenken4.txt' (7x7).

`2` run cell by cell

`3` Solutions are towards to end of this notebook


In [1]:
'''
    Input: path of a .txt file contains a kenken puzzle
    Output: 
        n: size of puzzle
        letter_cells: puzzle cages
            example:ABBCDD
                    AEECFD
                    GGHHFD
                    GGIJKK
                    LLIJJM
                    NNNOOM
        constrains: a dictionary of arithmetic constrains of the kenken puzzle
                    example showed below.
'''
def readkenken(path):
    kenken = open(path,'r')
    z = kenken.read()
    kenken.close()
    n = int(z[0])
    z = z.split('\n')
    cells = z[1:n+1]

    constrains = {}
    for c in z[n+1:]:
        
        line = c.split(':')
        letter = line[0]
        if line[1][-1] in '+-*/':
            num = int(line[1][:-1])
            sign = line[1][-1]
        else:
            num = int(line[1])
            sign = 'c' # constant constrain
        constrains[letter] = {'target':num,
                             'oper':sign,
                             'cell':[]}
    for x, line in enumerate(cells):
        for y, letter in enumerate(line):
            coor = (x,y)
            constrains[letter]['cell'].append(coor)
    return n,cells,constrains

In [53]:
import copy
import numpy as np
import math
import random

def print_factors(x, size):
    #returns a factors of x less than size
        fac = []
        for i in range(1, size+1):
            if x % i == 0:
                fac.append(i)
        return fac
    

'''Cell represent a cell in a kenken puzzle
    args:
        row: row number
        col: column number
        values: initial all possible values
            example: for 6*6 puzzle, all cell has initial value [1,2,3,4,5,6] '''
class Cell:
    def __init__(self, row, col, values):
        self.row = row
        self.col = col
        self.possible_val = copy.copy(values)
        self.last_possible_val = copy.copy(values)
        self.constrains = []
        self.value = 0
        self.curr_possible_val = []
    def getValues(self):
        return self.possible_val
    def getRow(self):
        return self.row
    def getCol(self):
        return self.col
    def getCurrPossibleVal(self):
        return self.curr_possible_val
    def resetValues(self):
        self.cur_values = copy.copy(last_possible_val)
    
    
'''RowColConstrain represents either a row or a column constrain of a kenken puzzle
    args:
        cells: a list of cells of a row or a column'''    
class RowColConstrain:
    def __init__(self, cells):
        self.cells = cells
        self.isArith = False
    def satisfy(self,values):
        #false if duplicates exsists
        #true otherwise
        return len(set(values))==len(values)
    def getCells(self):
        return self.cells
    
    
'''ArithConstrain represents an arithmetic constrain of a kenken puzzle
    args:
        target: target value
        cells: a list of cells involved in this arithmetic constrain
        oper: arithmetic operator '+-*/c'
        puzzle: Puzzle class
            example: A:120*, then target is 120 and operator is '*' '''
class ArithConstrain:
    def __init__(self,target,cells,oper,puzzle):
        self.isArith = True
        self.target = target
        self.cells = cells
        self.oper = oper
        self.size = puzzle.size
        self.domain = [i+1 for i in range(puzzle.size)]
        self.limitDomain()
    def getCells(self):
        return self.cells
    def satisfy(self,values):
        #checks if a certain combination of cell values satisfy an arithmetic constrain
        #also returns true if any cell is not assigned in the constrain
        
        values = np.array(values)
        if self.oper == '+':
            if len(values[values!=0]) < len(self.cells):
                return np.sum(values)<self.target
            return np.sum(values) == self.target
        elif self.oper == '*':
            if len(values[values!=0]) < len(self.cells):
                if len(values[values!=0]) == 0:
                    return True
                return self.target % np.prod(values) == 0
            return np.prod(values) == self.target
        elif self.oper == '-':
            if len(values[values!=0]) < len(self.cells):
                return True
            return abs(values[0]-values[1]) == self.target
        elif self.oper == '/':
            if len(values[values!=0]) < len(self.cells):
                if len(values[values!=0]) == 0:
                    return True
                return (int(values[values!=0][0] * self.target) in self.domain or (int(values[values!=0][0]/self.target) in self.domain and values[values!=0][0] % self.target==0) )
            larger = values[0] if values[0]>values[1] else values[1]
            smaller = values[1] if values[0]>values[1] else values[0]
            if larger % smaller != 0:
                return False
            else:
                return int(larger/smaller) == self.target
        elif self.oper == 'c':
            if len(values[values!=0]) == 0: return True
            return values[0]== self.target
        else:
            raise NameError('incorrect operator: ' + self.oper)

    def limitDomain(self):
        #reduce the possible values of cells in an arithmetics constrain based on arithmetic operator
        if self.oper == '+':
            if len(self.cells) == 2:
                if self.target <= self.size:
                    self.domain = list(range(1,self.target+1))
                elif self.target > self.size:
                    self.domain = list(range(self.target-self.size,self.size+1))
        elif self.oper == '*':
            factors = print_factors(self.target, self.size)
            if len(self.cells)<2:
                raise ValueError('incorrect numbers for multiplication',len(self.cells))
            elif len(self.cells)>2:
                self.domain = factors
            else:
                domain_set = set()
                for i in range(1,self.size+1):
                    for j in range(1,i+1):
                        if i*j == self.target:
                            domain_set.add(i)
                            domain_set.add(j)
                self.domain = list(domain_set)
        elif self.oper == '-':
            if len(self.cells) != 2:
                raise ValueError('incorrect numbers for subtraction',len(self.cells))
            domain_set = set()
            for i in range(1,self.size+1):
                for j in range(1,i+1):
                    if i - j == self.target:
                        domain_set.add(i)
                        domain_set.add(j)
            self.domain = list(domain_set)
        elif self.oper == '/':
            if len(self.cells) != 2:
                raise ValueError('incorrect numbers for division',len(self.cells))
            domain_set = set()
            for i in range(1,self.size+1):
                for j in range(1,i+1):
                    if i % j == 0 and int(i/j)==self.target:
                        domain_set.add(i)
                        domain_set.add(j)
            self.domain = list(domain_set)
        elif self.oper == 'c':
            self.domain = [self.target,]
        else:
            raise NameError('incorrect operator: ' + self.oper)
            
    
'''Puzzle represents a kenken puzzle
    args: 
        size: size of puzzle. Ex: for 6*6 puzzle, size = 6
        arith_dict: a dictionary of arithmetic constrain
        init: True - then generate random answer for local search
              False - assign 0 to all cels
          '''
class Puzzle:
    def __init__(self, size,arith_dict,init=True):
        self.size = size
        self.domain = [i+1 for i in range(size)]
        self.cells = []
        self.constrains =[] # constrain list
        for row in range(size):
            for col in range(size):
                self.cells.append(Cell(row,col,self.domain))
                
        self.addRowColConstrains()
        #add RowColconstrains to cells
        for constrain in self.constrains:
            for cell in constrain.getCells():
                cell.constrains.append(constrain)
                
        self.addArithConstrains(arith_dict)
        #assigning neighbors
        self.neighbors = {} 
        for cell in self.cells:
            n = []
            val = []
            for s in range(size):
                if s != cell.row:
                    n.append(self.getCell(s, cell.col))
                if s != cell.col:  
                    n.append(self.getCell(cell.row, s))
            self.neighbors[cell] = n
        if init: 
            # randomly assign values to all cells that have the correct number of each number
            # Only use in local search
            self.initializeValue()
    def getCells(self):
        return self.cells
    def getCell(self,row,col):
        return self.cells[row*self.size+col]
    def getRow(self,row):
        return self.cells[row*self.size:(row+1)*self.size]
    def getCol(self,col):
        cells = []
        for row in range(self.size):
            cells.append(self.cells[row*self.size+col])
        return cells
    def addRowColConstrains(self):
        for i in range(self.size):
            self.constrains.append(RowColConstrain(self.getRow(i)))
            self.constrains.append(RowColConstrain(self.getCol(i)))
    def addArithConstrains(self,arith_dict):
        for key, value in arith_dict.items():
            target = int(value['target'])
            oper = value['oper']
            cells = []
            for cell in value['cell']:
                row = cell[0]
                col = cell[1]
                cells.append(self.getCell(row,col))
            constrain = ArithConstrain(target,cells,oper,self)
            self.constrains.append(constrain)
            # add arithmetic constrains to cells
            for cell in cells:
                cell.constrains.append(constrain)
                cell.possible_val = copy.copy(constrain.domain)
                cell.curr_possible_val = copy.copy(constrain.domain)
    def getValuesArray(self):
        #return a 2d numpy array representation of the puzzle
        arr = np.zeros((self.size,self.size),dtype=int)
        for row in range(self.size):
            col = 0
            for cell in self.getRow(row):
                arr[row][col] = cell.value
                col = col+1
        return arr
    def printPuzzle(self):
        #print puzzle
        print('-'*20)
        print(self.getValuesArray())
        print('-'*20)
    def getNeighborVals(self,cell):
        #return a list of neighbors (all cells in the same row and in the same column) of a cell in the puzzle
        neighborVals = []
        for n in self.neighbors[cell]:
            if n.value != 0:
                neighborVals.append(n.value)
        return neighborVals
    def getUnassigned(self):
        #return a list of unassigned cells
        unassigned = []
        for index, cell in enumerate(self.getCells()):
            if cell.value == 0:
                unassigned.append(cell)
        return unassigned
    def initializeValue(self):
        # randomly assign values to all cells that have the correct number of each number
        a= np.array(np.arange(self.size*self.size))
        np.random.shuffle(a)
        a=a%self.size + 1
        for i in range(len(a)):
            self.cells[i].value = a[i]

 * **Solutions**
 
 Simple BackTrack
 
 Minimum Remaining Value BackTrack
 
 Local Search (Hill climbing and tabu search)

In [54]:
'''Simple BackTrack search assign values left-to-right then top-to-bottom.
    args:
        puzzle: a Puzzle class to solve'''
class simpleBackTrack:
    def __init__(self,puzzle):
        self.puzzle = puzzle
        self.n=0 # number of iterations
        
    def isSolved(self):
        #Check if puzzle is solved
        # satisfy all constrains and all cells are assigned
        for constrain in self.puzzle.constrains:
            possible_value = []
            for cell in constrain.getCells():
                if cell.value == 0:
                    return False
                possible_value.append(cell.value)
            if not constrain.satisfy(possible_value):
                return False
        return True
    
    def isNoValue(self):
        #Check if reaches a deadend (no possible values)
        for constrain in self.puzzle.constrains:
            possible_value = []
            for cell in constrain.getCells():
                if cell.value == 0:
                    break
                possible_value.append(cell.value)
            if not constrain.satisfy(possible_value):
                return True
        return False

    def solve(self):
        def helper(index):
            if index >= self.puzzle.size ** 2:
                # reaches the end
                return True
            self.n+=1
#             self.puzzle.printPuzzle()
            if self.isSolved():
                self.puzzle.printPuzzle()
                return True
            elif self.isNoValue():
                return False
            else:
                cell = self.puzzle.getCells()[index]
                row = math.floor(index/self.puzzle.size)
                col = index % self.puzzle.size
                arr = self.puzzle.getValuesArray()
                cell_possible=[] # possible val limited by arithmetic constrain
                for each in cell.possible_val:
                    if not each in arr[row,:] and not each in arr[:,col]:
                        cell_possible.append(each)
                if len(cell_possible) == 0:
                    return False
#                 print(row, col ,cell_possible)
                for value in cell_possible:
                    cell.value = value
                    if helper(index+1):
                        return True
                    else:
                        cell.value = 0
                return False
        return helper(0)

In [55]:
'''BackTrack search using minimum remain value
    args:
        puzzle: a Puzzle class to solve'''
class MRVBackTrack:
    def __init__(self,puzzle):
        self.puzzle = puzzle
        self.n=0 # num of iterations
    def isSolved(self):
        #check a puzzle satisfies all constrains and all cells are assigned
        for constrain in self.puzzle.constrains:
            possible_value = []
            for cell in constrain.getCells():
                if cell.value == 0:
                    return False
                possible_value.append(cell.value)
            if not constrain.satisfy(possible_value):
                return False
        return True
    def isNoValue(self):
        #check if reaches deadend
        for constrain in self.puzzle.constrains:
            
            possible_value = []
            for cell in constrain.getCells():
                
                if cell.value == 0:
                    break
                possible_value.append(cell.value)
            if not constrain.satisfy(possible_value):
                return True
        return False
    def solve(self):
        def helper(index):
            if index >= self.puzzle.size ** 2:
                return True
            self.n+=1
            #self.puzzle.printPuzzle()
            if self.isSolved():
                self.puzzle.printPuzzle()
                return True
            elif self.isNoValue():
                return False
            else:
                cell = self.puzzle.getCells()[index]
                if len(cell.curr_possible_val) == 0:
                    return False
#                 print(row, col ,cell_possible)
                for value in cell.curr_possible_val:
                    self.assign_val(cell, value)
                    if helper(self.find_min_remain()):
                        return True
                    else:
                        self.assign_val(cell, 0)
                return False
        return helper(self.find_min_remain())
    def find_min_remain(self):
        if len(self.puzzle.getUnassigned()) == 0:
            return self.puzzle.size ** 2 # all assigned
        # sort unassigned cells based on length of possible values.
        sortedList = sorted(self.puzzle.getUnassigned(), key = lambda x: len(x.getCurrPossibleVal()))
        return (self.puzzle.size * sortedList[0].getRow()) + sortedList[0].getCol()
    def assign_val(self, cell, value):
        temp = cell.value
        cell.value = value
        if value == 0:
            # assign 0 to current cell
            # add back value to possible value of neighbor cells if originally exist
            for nb in self.puzzle.neighbors[cell]:
                if (temp not in nb.curr_possible_val) and (temp in nb.possible_val) and (temp not in self.puzzle.getNeighborVals(nb)):
                    nb.curr_possible_val.append(temp)              
        else: 
            # remove assigned value from neighbor cells (all cells in the same row and same col)
            for nb in self.puzzle.neighbors[cell]:
                if value in nb.curr_possible_val:
                    #print(nb.row, nb.col," removing ",value)
                    nb.curr_possible_val.remove(value)
        
                            
    

In [64]:
'''Puzzle is initialized with correct number of each numbers.
Then values of two cells are switched base on minimizing the 
number of conflicts in the puzzle. Conflict are the sum of 
duplicates in rows and cols, and the number of cells that do not
satisfy their arithmetic constrains.
If a puzzle reaches a local minimum, meaning switching any of the two
cells in the puzzle would INCREASE the number of conflict, then the 
puzzle is randomly initialized again util finding a solution (conflict == 0)
or reach 500 iterations.
A tabu list of 25 is used to avoid looping among states have the save
number of conflicts.'''
class localSearch:
    def __init__(self,puzzle):
        self.puzzle = puzzle
        self.n=0
        self.limit = 500
        self.tabu = []
    
    def findConflict(self,puzzle):
        
       #arith constrain: not satisfied constrain
       #row col constrain: repeated numbers in each row and col

        conflict = 0
        arr = puzzle.getValuesArray()
        for i in range(puzzle.size):
            conflict = conflict + puzzle.size-len(set(arr[i,:]))
            conflict = conflict + puzzle.size-len(set(arr[:,i]))
        for constrain in puzzle.constrains:
            if constrain.isArith:
                val = []
                for cell in constrain.cells:
                    val.append(cell.value)
                if not constrain.satisfy(val):
                    conflict = conflict+len(constrain.cells)
        return conflict
    
    def solve(self):
        while True:
            if self.n > self.limit:
                print('unable to solve')
                break
            if self.findConflict(self.puzzle) == 0:
                print('solved in {:d} number of iterations'.format(self.n))
                self.puzzle.printPuzzle()
                break
            conflict_min = self.findConflict(self.puzzle)
            cell_min = (-1,-1)
            #iterate through all cells to find a pair of cells for switching that minimizes
            #the number of conlficts in the puzzle
            for n in range(len(self.puzzle.cells)):
                for j in range(n,len(self.puzzle.cells)):
                    newpuzzle = copy.deepcopy(self.puzzle) #deepcopy to makesure the original puzzle is not changed
                    newpuzzle.cells[n].value = self.puzzle.cells[j].value
                    newpuzzle.cells[j].value = self.puzzle.cells[n].value
                    if newpuzzle.getValuesArray().tolist() not in self.tabu:
                        conf = self.findConflict(newpuzzle)
                        if conf <= conflict_min:
                            cell_min = (n,j)
                            conflict_min = conf

            if cell_min == (-1,-1):
                self.puzzle.initializeValue()
            else:
                # a pair of cell is found, switch, append to tabu list
                n_val = self.puzzle.cells[cell_min[0]].value
                self.puzzle.cells[cell_min[0]].value = self.puzzle.cells[cell_min[1]].value
                self.puzzle.cells[cell_min[1]].value = n_val
                self.tabu.append(self.puzzle.getValuesArray().tolist())
                if len(self.tabu) > 25:
                    self.tabu.pop(0)
            self.n+=1
            print('Iteration: ',self.n,' |||| # of conflicts: ', self.findConflict(self.puzzle))
#             self.puzzle.printPuzzle()    

kenken.txt: 6 by 6 given in HW 1

kenken2.txt: 5 by 5

kenken3.txt: 4 by 4

kenken4.txt: 7 by 7

feel free to use any of these to test

In [60]:
n,letter_cells,constrains = readkenken('kenken.txt')
for key, value in constrains.items():
    print(key, value)

A {'target': 11, 'oper': '+', 'cell': [(0, 0), (1, 0)]}
B {'target': 2, 'oper': '/', 'cell': [(0, 1), (0, 2)]}
C {'target': 20, 'oper': '*', 'cell': [(0, 3), (1, 3)]}
D {'target': 6, 'oper': '*', 'cell': [(0, 4), (0, 5), (1, 5), (2, 5)]}
E {'target': 3, 'oper': '-', 'cell': [(1, 1), (1, 2)]}
F {'target': 3, 'oper': '/', 'cell': [(1, 4), (2, 4)]}
G {'target': 240, 'oper': '*', 'cell': [(2, 0), (2, 1), (3, 0), (3, 1)]}
H {'target': 6, 'oper': '*', 'cell': [(2, 2), (2, 3)]}
I {'target': 6, 'oper': '*', 'cell': [(3, 2), (4, 2)]}
J {'target': 7, 'oper': '+', 'cell': [(3, 3), (4, 3), (4, 4)]}
K {'target': 30, 'oper': '*', 'cell': [(3, 4), (3, 5)]}
L {'target': 6, 'oper': '*', 'cell': [(4, 0), (4, 1)]}
M {'target': 9, 'oper': '+', 'cell': [(4, 5), (5, 5)]}
N {'target': 8, 'oper': '+', 'cell': [(5, 0), (5, 1), (5, 2)]}
O {'target': 2, 'oper': '/', 'cell': [(5, 3), (5, 4)]}


* **Simple BackTrack**

In [61]:
import time
kenken = Puzzle(n,constrains,init=False)
simplebacktrack = simpleBackTrack(kenken)
st = time.time()
simplebacktrack.solve()
et = time.time()
kenken.printPuzzle()
print(simplebacktrack.n,"iterations")
print('in {:f} seconds'.format(et-st))

--------------------
[[5 6 3 4 1 2]
 [6 1 4 5 2 3]
 [4 5 2 3 6 1]
 [3 4 1 2 5 6]
 [2 3 6 1 4 5]
 [1 2 5 6 3 4]]
--------------------
497 iterations
in 0.103325 seconds


* **MRV BackTrack**

In [62]:
kenken = Puzzle(n,constrains,init=False)
MRVsolver = MRVBackTrack(kenken)
st = time.time()
MRVsolver.solve()
et = time.time()
MRVsolver.puzzle.printPuzzle()
print(MRVsolver.n, "iterations")
print('in {:f} seconds'.format(et-st))

--------------------
[[5 6 3 4 1 2]
 [6 1 4 5 2 3]
 [4 5 2 3 6 1]
 [3 4 1 2 5 6]
 [2 3 6 1 4 5]
 [1 2 5 6 3 4]]
--------------------
79 iterations
in 0.016597 seconds


* **Local Search**
a 6*6 puzzle takes a long time to solve.

It has possibility to be trapped in a loop when the number of possible switches having the same number of conflict exceeds the length of tabu list.

It also has a hard time converge to the correct solution if the puzzle is large.

Here, we use a 5*5 puzzle found on kenkenpuzzle.com to demonstrate.

In [68]:
# a 5*5 puzzle
n,letter_cells,constrains = readkenken('kenken2.txt')#comment this line if want to see the original puzzle
# n,letter_cells,constrains = readkenken('kenken.txt')#uncomment this line if want to see the original puzzle
print('puzzle size: {:d} by {:d}'.format(n,n))
kenken = Puzzle(n,constrains,init=True)
LSsolver = localSearch(kenken)
LSsolver.solve()

puzzle size: 5 by 5
Iteration:  1  |||| # of conflicts:  30
Iteration:  2  |||| # of conflicts:  24
Iteration:  3  |||| # of conflicts:  21
Iteration:  4  |||| # of conflicts:  18
Iteration:  5  |||| # of conflicts:  16
Iteration:  6  |||| # of conflicts:  13
Iteration:  7  |||| # of conflicts:  12
Iteration:  8  |||| # of conflicts:  11
Iteration:  9  |||| # of conflicts:  9
Iteration:  10  |||| # of conflicts:  6
Iteration:  11  |||| # of conflicts:  4
Iteration:  12  |||| # of conflicts:  4
Iteration:  13  |||| # of conflicts:  2
Iteration:  14  |||| # of conflicts:  2
Iteration:  15  |||| # of conflicts:  2
Iteration:  16  |||| # of conflicts:  34
Iteration:  17  |||| # of conflicts:  28
Iteration:  18  |||| # of conflicts:  22
Iteration:  19  |||| # of conflicts:  19
Iteration:  20  |||| # of conflicts:  16
Iteration:  21  |||| # of conflicts:  14
Iteration:  22  |||| # of conflicts:  11
Iteration:  23  |||| # of conflicts:  9
Iteration:  24  |||| # of conflicts:  8
Iteration:  25